In [1]:
pip install tensorflow-recommenders

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.2/96.2 kB 1.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 23.5 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: keras
    Found existing installation: keras 3.3.3
    Uninstalling keras-3.3.3:
      Successfully uninstalled keras-3.3.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-decision-forests 1.8.1 requires wurlitzer, which is not installed.
Note: you may need to restart the kernel to use updated packages.


In [3]:
#pip install scann

In [2]:
import tensorflow_recommenders as tfrs
import tensorflow as tf
import pandas as pd
import numpy as np
from typing import Dict, Text
import tempfile
import os

2024-06-09 17:51:29.872222: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-09 17:51:29.872387: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-09 17:51:30.017152: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
similar_df = pd.read_excel('/kaggle/input/recommender-system/Data_RecomenderSystem.xlsx', sheet_name='similar')
mentor_df = pd.read_excel('/kaggle/input/recommender-system/Data_RecomenderSystem.xlsx', sheet_name='mentor')

In [4]:
similar_df.head()

,user_id,mentor_id,kriteria_mentor_user,kriteria_mentor,provinsi_user,kabupaten_kota_user,ketertarikan_user,similarity
0,1,1,Saya ingin belajar fisika dengan mahasiswa fis...,Saya mahasiswa fisika Universitas Gadjah Mada ...,Daerah Istimewa Yogyakarta,Sleman,Fisika dan Matematika,5
1,1,2,Saya ingin belajar fisika dengan mahasiswa fis...,Saya berpengalaman dalam mengajar fisika dan m...,Daerah Istimewa Yogyakarta,Sleman,Fisika dan Matematika,4
2,1,3,Saya ingin belajar fisika dengan mahasiswa fis...,Saya menggunakan beberapa warna dalam mencatat...,Daerah Istimewa Yogyakarta,Sleman,Fisika dan Matematika,2
3,1,17,Saya ingin belajar fisika dengan mahasiswa fis...,Mengajar matematika untuk SMA dengan pengalama...,Daerah Istimewa Yogyakarta,Sleman,Fisika dan Matematika,3
4,1,22,Saya ingin belajar fisika dengan mahasiswa fis...,Berpengalaman mengajar matematika untuk anak S...,Daerah Istimewa Yogyakarta,Sleman,Fisika dan Matematika,3


In [5]:
mentor_df.head()

,mentor_id,jenis_kelamin_mentor,pendidikan_terakhir_mentor,provinsi_mentor,kabupaten_kota_mentor,kemampuan_mentor,bahasa_mentor,kriteria_mentor
0,1,Laki-laki,S1,Daerah Istimewa Yogyakarta,Sleman,"Kalkulus, Fisika",Indonesia,Saya mahasiswa fisika Universitas Gadjah Mada ...
1,2,Laki-laki,S1,Daerah Istimewa Yogyakarta,Sleman,Fisika,Indonesia,Saya berpengalaman dalam mengajar fisika dan m...
2,3,Perempuan,S1,Daerah Istimewa Yogyakarta,Sleman,Fisika,Indonesia,Saya menggunakan beberapa warna dalam mencatat...
3,4,Perempuan,S1,Daerah Istimewa Yogyakarta,Sleman,Hukum,Indonesia,Saya sedang menekuni kuliah sarjana strata 1 h...
4,5,Laki-laki,S1,Daerah Istimewa Yogyakarta,Sleman,sosiologi,Indonesia,saya sangat mendalami pelajaran sosiologi saat...


In [6]:
mentor_df['mentor_id'] = mentor_df['mentor_id'].astype(str)
similar_df['mentor_id'] = similar_df['mentor_id'].astype(str)

similar = tf.data.Dataset.from_tensor_slices(dict(similar_df[['kriteria_mentor_user', 'mentor_id', 'similarity']]))
mentor = tf.data.Dataset.from_tensor_slices(dict(mentor_df[['mentor_id']]))

similar = similar.map(lambda x: {
    'mentor_id': x['mentor_id'],
    'kriteria_mentor_user': x['kriteria_mentor_user'],
    'similarity': float(x['similarity'])
})

mentor = mentor.map(lambda x: x['mentor_id'])

In [7]:
train_split = 0.7
len_data = similar_df.shape[0]
split_size = int(len_data*train_split)

tf.random.set_seed(42)
shuffled = similar.shuffle(1000, seed=42, reshuffle_each_iteration=False)

train = shuffled.take(split_size)
test = shuffled.skip(split_size).take(len_data-split_size)

In [8]:
mentor_id = mentor.batch(32)
kriteria_mentor_user = similar.batch(8).map(lambda x: x['kriteria_mentor_user'])

unique_mentor_id = np.unique(np.concatenate(list(mentor_id)))
unique_kriteria_mentor_user = np.unique(np.concatenate(list(kriteria_mentor_user)))

unique_kriteria_mentor_user[:5]

array([b'Dicari orang untuk acara seminar yang ahli data science secara online terutama untuk yang Non STEM dapat menjelaskannya dengan sangat sederhana',
       b'Memiliki penguasaan materi yang mendalam dan mampu menjelaskan konsep-konsep yang kompleks dengan cara yang mudah dipahami',
       b'Mencari mentor Kimia yang Energetik, kreatif, dan selalu memberikan dorongan positif kepada siswa dalam proses pembelajaran',
       b'Mencari mentor bahasa jepang yang praktis dan berpengalaman',
       b'Minat di bisnis makanan, mencari mentor yang sangat berpengalaman dalam bisnis makanan'],
      dtype=object)

In [9]:
embedding_dimension = 32

In [10]:
user_model = tf.keras.Sequential([
  tf.keras.layers.StringLookup(
      vocabulary=unique_kriteria_mentor_user, mask_token=None),
  # We add an additional embedding to account for unknown tokens.
  tf.keras.layers.Embedding(len(unique_kriteria_mentor_user) + 1, embedding_dimension)
])

In [11]:
mentor_model = tf.keras.Sequential([
    tf.keras.layers.StringLookup(
    vocabulary=unique_mentor_id, mask_token=None),
    tf.keras.layers.Embedding(len(unique_mentor_id)+1, embedding_dimension)
])

In [12]:
metrics = tfrs.metrics.FactorizedTopK(
  candidates=mentor.batch(128).map(mentor_model)
)

In [13]:
task = tfrs.tasks.Retrieval(
  metrics=metrics
)

In [14]:
class MentorModel(tfrs.Model):
    def __init__(self, user_model, mentor_model):
        super().__init__()
        self.mentor_model: tf.keras.Model = mentor_model
        self.user_model: tf.keras.Model = user_model
        self.task: tf.keras.layers.Layer = task

    def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
    # We pick out the user features and pass them into the user model.
        user_embeddings = self.user_model(features["kriteria_mentor_user"])
        # And pick out the kriteria_mentor features and pass them into the mentor model,
        # getting embeddings back.
        positive_mentor_embeddings = self.mentor_model(features["mentor_id"])

        # The task computes the loss and the metrics.
        return self.task(user_embeddings, positive_mentor_embeddings)

In [15]:
class NoBaseClassMentorModel(tf.keras.Model):
    def __init__(self, user_model, mentor_model):
        super().__init__()
        self.mentor_model: tf.keras.Model = mentor_model
        self.user_model: tf.keras.Model = user_model
        self.task: tf.keras.layers.Layer = task
    
    def train_step(self, features: Dict[Text, tf.Tensor]) -> tf.Tensor:
        with tf.GradientTape() as tape:
            user_embeddings = self.user_model(features["kriteria_mentor_user"])
            positive_mentor_embeddings = self.mentor_model(features['mentor_id'])
            loss = self.task(user_embeddings, positive_mentor_embeddings)
            
            regularization_loss = sum(self.losses)

            total_loss = loss + regularization_loss

        gradients = tape.gradient(total_loss, self.trainable_variables)
        self.optimizer.apply_gradients(zip(gradients, self.trainable_variables))

        metrics = {metric.name: metric.result() for metric in self.metrics}
        metrics["loss"] = loss
        metrics["regularization_loss"] = regularization_loss
        metrics["total_loss"] = total_loss

        return metrics
    
    def test_step(self, features: Dict[Text, tf.Tensor]) -> tf.Tensor:

        # Loss computation.
        user_embeddings = self.user_model(features["kriteria_mentor_user"])
        positive_mentor_embeddings = self.mentor_model(features["mentor_id"])
        loss = self.task(user_embeddings, positive_mentor_embeddings)

        # Handle regularization losses as well.
        regularization_loss = sum(self.losses)

        total_loss = loss + regularization_loss

        metrics = {metric.name: metric.result() for metric in self.metrics}
        metrics["loss"] = loss
        metrics["regularization_loss"] = regularization_loss
        metrics["total_loss"] = total_loss

        return metrics

In [16]:
model = MentorModel(user_model=user_model, mentor_model=mentor_model)
model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.1))

In [17]:
cached_train = train.shuffle(100).batch(4).cache()
cached_test = test.batch(2).cache()

In [18]:
cached_train

<CacheDataset element_spec={'mentor_id': TensorSpec(shape=(None,), dtype=tf.string, name=None), 'kriteria_mentor_user': TensorSpec(shape=(None,), dtype=tf.string, name=None), 'similarity': TensorSpec(shape=(None,), dtype=tf.float32, name=None)}>

In [19]:
model.fit(cached_train, epochs=30)

Epoch 1/30
8/8 [==============================] - 2s 45ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_10_categorical_accuracy: 0.0323 - factorized_top_k/top_50_categorical_accuracy: 0.0323 - factorized_top_k/top_100_categorical_accuracy: 0.1613 - loss: 5.0477 - regularization_loss: 0.0000e+00 - total_loss: 5.0477 
Epoch 2/30
8/8 [==============================] - 0s 45ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.2903 - factorized_top_k/top_10_categorical_accuracy: 0.4516 - factorized_top_k/top_50_categorical_accuracy: 0.8387 - factorized_top_k/top_100_categorical_accuracy: 0.9032 - loss: 5.0023 - regularization_loss: 0.0000e+00 - total_loss: 5.0023
Epoch 3/30
8/8 [==============================] - 0s 46ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.709

In [20]:
model.evaluate(cached_test, return_dict=True)

7/7 [==============================] - 1s 45ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0714 - factorized_top_k/top_10_categorical_accuracy: 0.0714 - factorized_top_k/top_50_categorical_accuracy: 0.1429 - factorized_top_k/top_100_categorical_accuracy: 0.3571 - loss: 1.2403 - regularization_loss: 0.0000e+00 - total_loss: 1.2403     


{'factorized_top_k/top_1_categorical_accuracy': 0.0,
 'factorized_top_k/top_5_categorical_accuracy': 0.0714285746216774,
 'factorized_top_k/top_10_categorical_accuracy': 0.0714285746216774,
 'factorized_top_k/top_50_categorical_accuracy': 0.1428571492433548,
 'factorized_top_k/top_100_categorical_accuracy': 0.3571428656578064,
 'loss': 0.8011033535003662,
 'regularization_loss': 0,
 'total_loss': 0.8011033535003662}

In [22]:
# Create a model that takes in raw query features, and
index = tfrs.layers.factorized_top_k.BruteForce(model.user_model)
# recommends movies out of the entire movies dataset.
index.index_from_dataset(
  tf.data.Dataset.zip((mentor.batch(100), mentor.batch(100).map(model.mentor_model)))
)

# Get recommendations.
_, titles = index(tf.constant(["Dicari orang untuk acara seminar yang ahli data science secara online terutama untuk yang Non STEM dapat menjelaskannya dengan sangat sederhana"]))
print(f"mentor_id: {titles[0, :3]}")

mentor_id: [b'8' b'245' b'20']


In [28]:
path = os.path.join("models/1")

  # Save the index.
tf.saved_model.save(index, path)

  # Load it back; can also be done in TensorFlow Serving.
loaded = tf.saved_model.load(path)

# Pass a user id in, get top predicted movie titles back.
scores, mentor_id = loaded(["Saya ingin belajar fisika dengan mahasiswa fisika yang sudah berpengalaman menjadi asisten dosen dan juga di daerah Yogyakarta dengan menggunakan bahasa indonesia harus laki-laki."])

print(f"Recommendations: {mentor_id[0][:3]}")

Recommendations: [b'17' b'3' b'1']


In [25]:
!zip -r file.zip /kaggle/working

  adding: kaggle/working/ (stored 0%)
  adding: kaggle/working/models/ (stored 0%)
  adding: kaggle/working/models/1/ (stored 0%)
  adding: kaggle/working/models/1/assets/ (stored 0%)
  adding: kaggle/working/models/1/saved_model.pb (deflated 84%)
  adding: kaggle/working/models/1/fingerprint.pb (stored 0%)
  adding: kaggle/working/models/1/variables/ (stored 0%)
  adding: kaggle/working/models/1/variables/variables.index (deflated 33%)
  adding: kaggle/working/models/1/variables/variables.data-00000-of-00001 (deflated 56%)
  adding: kaggle/working/.virtual_documents/ (stored 0%)


In [26]:
!ls

file.zip  models


In [27]:
from IPython.display import FileLink
FileLink(r'file.zip')

/kaggle/working/file.zip